# Caculate ADX.
Ref: https://blog.quantinsti.com/adx-indicator-python/

In [10]:
from vnstock import *
from datetime import date, datetime, timedelta
import numpy as np

def fm_date(date):
    return date.strftime("%Y-%m-%d")

# Get list of dates for the last 365 days
def get_last_days(num_days):
    current_date = datetime.now()
    days_ago = current_date - timedelta(days=num_days)
    date_list = [days_ago + timedelta(days=i) for i in range(num_days)]
    # Format the dates as "%Y-%m-%d" and store them in a new list
    formatted_dates = [fm_date(date) for date in date_list]

    return formatted_dates

def get_stock_price(symbol, num_days):
    dates_train = get_last_days(num_days)
    df_his = stock_historical_data(symbol, dates_train[0], dates_train[len(dates_train) - 1], '1D')
    return df_his


data = stock_historical_data(symbol='TCB', start_date='2022-06-01', end_date='2023-06-17', resolution='1D', type='stock')

In [11]:
print(data.head())

         time   open   high    low  close   volume ticker
0  2022-06-01  37000  37700  36650  36850  3829000    TCB
1  2022-06-02  36900  37000  36000  36400  3533700    TCB
2  2022-06-03  36350  36850  36050  36450  3040200    TCB
3  2022-06-06  36450  36800  36000  36050  3845100    TCB
4  2022-06-07  36000  36200  35000  36200  4640800    TCB


In [12]:
def calculate_adx(df, period=14):
    """
    Calculate the Average Directional Index (ADX) using the given high, low, and close prices.
    
    Parameters:
    - high: High prices for each period (list or array)
    - low: Low prices for each period (list or array)
    - close: Close prices for each period (list or array)
    - period: Number of periods to consider (default is 14)
    
    Returns:
    - adx: The calculated ADX values
    """
    
    # Calculate True Range (TR)
    df['High-Low'] = df['high'] - df['low']
    df['High-PrevClose'] = abs(df['high'] - df['close'].shift(1))
    df['Low-PrevClose'] = abs(df['low'] - df['close'].shift(1))
    df['TrueRange'] = df[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)
    
    # Calculate Directional Movement (DM)
    df['High-PrevHigh'] = df['high'] - df['high'].shift(1)
    df['PrevLow-Low'] = df['low'].shift(1) - df['low']
    
    df['PlusDM'] = df.apply(lambda row: row['High-PrevHigh'] if row['High-PrevHigh'] > row['PrevLow-Low'] and row['High-PrevHigh'] > 0 else 0, axis=1)
    df['MinusDM'] = df.apply(lambda row: row['PrevLow-Low'] if row['PrevLow-Low'] > row['High-PrevHigh'] and row['PrevLow-Low'] > 0 else 0, axis=1)
    
    # Smoothed True Range (ATR)
    df['ATR'] = df['TrueRange'].rolling(window=period).mean()
    
    # Smoothed Plus Directional Movement (+DI)
    df['SmoothedPlusDM'] = df['PlusDM'].rolling(window=period).mean()
    
    # Smoothed Minus Directional Movement (-DI)
    df['SmoothedMinusDM'] = df['MinusDM'].rolling(window=period).mean()
    
    # Calculate Plus Directional Indicator (+DI) and Minus Directional Indicator (-DI)
    df['PlusDI'] = (df['SmoothedPlusDM'] / df['ATR']) * 100
    df['MinusDI'] = (df['SmoothedMinusDM'] / df['ATR']) * 100
    
    # Calculate Directional Movement Index (DX)
    df['DX'] = (abs(df['PlusDI'] - df['MinusDI']) / (df['PlusDI'] + df['MinusDI'])) * 100
    
    # Calculate Average Directional Index (ADX)
    df['ADX'] = df['DX'].rolling(window=period).mean()
 
    return df


df_adx = calculate_adx(data, 14)

In [15]:
adx_values = df_adx['ADX'][13:].values
print(adx_values)

[        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
         nan 14.90214925 16.36582071 15.7817487  15.90498518 16.46553084
 16.90398932 16.93080333 16.48570388 16.41471658 15.97494809 16.85743351
 18.62873548 17.42492311 16.39406942 13.77855628 10.67610318 12.77117791
 16.30885146 19.84149331 23.42562256 26.90467466 30.85210338 35.4649074
 39.99609914 42.65000134 43.49507176 46.5857311  50.43513988 54.68056112
 57.83649016 57.54336348 55.81525288 53.29206178 50.87066079 49.03059868
 47.35464848 43.7231959  39.21653    35.06744314 32.75975083 30.03539185
 26.25441133 23.67174681 22.10311935 21.39263303 21.45503703 22.41265084
 23.90066352 26.8733526  30.07475171 35.00167901 40.75526752 47.11178603
 53.04381697 58.96966682 65.46450714 70.23235685 74.44481106 77.98238366
 82.04941265 86.41278816 90.23089792 91.87555579 93.30761532 93.88488971
 94.64978271 95.22419749 95.76025987 96.30195129 95.

In [22]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create layout
layout = go.Layout(title='Average Directional Index (ADX)',
                   xaxis=dict(title='Period'),
                   yaxis=dict(title='ADX'),
                   showlegend=True)

fig = make_subplots(rows=2, cols=1)

trace_price = go.Scatter(x=list(range(len(df_adx['close']))), y=df_adx['close'], mode='lines', name='Price')
# Create a trace for ADX values
trace = go.Scatter(x=list(range(len(df_adx['ADX']))), y=df_adx['ADX'], mode='lines', name='ADX')

# Create figure
fig.add_trace(trace_price, row=1, col=1)
fig.add_trace(trace, row=2, col=1)

# Show the figure
fig.show()